# 🔄 GRU：门控循环单元（Gated Recurrent Unit）

---

## 📌 一、为什么使用 GRU？

RNN 在面对长序列时，容易遇到两个问题：

- ❌ 梯度消失/爆炸，导致无法有效学习长期依赖；
- ❌ 每次状态完全覆盖，难以“记住”和“忘记”特定信息。

✅ GRU 引入了两种门机制：
- 更新门（update gate）：控制保留多少旧信息；
- 重置门（reset gate）：控制整合多少旧信息。

---

## 🧠 二、GRU 的公式结构（D2L 版本）

设：
- 输入为 $X_t \in \mathbb{R}^d$
- 上一隐藏状态为 $H_{t-1} \in \mathbb{R}^h$

GRU 结构如下：

### 1️⃣ 更新门（update gate）
$$
Z_t = \sigma(X_t W_{xz} + H_{t-1} W_{hz} + b_z)
$$

### 2️⃣ 重置门（reset gate）
$$
R_t = \sigma(X_t W_{xr} + H_{t-1} W_{hr} + b_r)
$$

### 3️⃣ 候选隐藏状态（candidate activation）
$$
\tilde{H}_t = \tanh(X_t W_{xh} + (R_t \odot H_{t-1}) W_{hh} + b_h)
$$

注意这里的 $R_t \odot H_{t-1}$ 表示：
- **根据重置门抑制旧状态的一部分信息**，再与当前输入组合

### 4️⃣ 当前隐藏状态（最终输出）
$$
H_t = (1 - Z_t) \odot H_{t-1} + Z_t \odot \tilde{H}_t
$$

含义：
- 更新门决定有多少保留旧状态，多少用新信息替代。

---


## 🏗️ 三、GRU 信息流结构图

               ┌─────────────┐
               │ xₜ          │
               └────┬────────┘
                    │
               ┌────▼────┐
               │ 拼接 [hₜ₋₁, xₜ] │
               └────┬────┘
    ┌──────────────┼──────────────┐
    ▼              ▼              ▼
┌──────┐       ┌──────┐      ┌─────────────┐
│ zₜ   │       │ rₜ   │      │ 重置状态 rₜ·hₜ₋₁ │
│sigmoid│       │sigmoid│      └─────────────┘
└──┬───┘       └──┬───┘             │
   │              │                ▼
   │              └────┐     ┌─────────────┐
   ▼                   └────▶│  候选状态 h̃ₜ  │
                          ┌──▶│ tanh(...)    │
                          │   └────┬────────┘
                          ▼        ▼
                  ┌─────────────────────────────┐
                  │ hₜ = (1 - zₜ)·hₜ₋₁ + zₜ·h̃ₜ │
                  └─────────────────────────────┘


---

## 🔁 四、GRU 与 LSTM 的对比

| 对比项       | GRU                          | LSTM                         |
|--------------|-------------------------------|------------------------------|
| 门控数量     | 2 个（更新门、重置门）        | 3 个（遗忘、输入、输出）      |
| 记忆结构     | 只有 $h_t$                   | 有 $h_t$ 和 $C_t$             |
| 参数数量     | 少                           | 多                           |
| 收敛速度     | 快                           | 略慢                         |
| 性能表现     | 多任务中性能不逊色            | 稳定性略优（长序列任务）       |

---

## 🧪 五、使用建议

- **GRU 更轻量、计算更快**，适合快速迭代和中短序列任务
- 如果你不确定用哪个，GRU 是个好起点
- 对长期依赖特别强的任务可优先考虑 LSTM

---

## 💬 常见 GRU 面试问题汇总与解答

---

### ❓1. GRU 有哪些门？各自的作用是什么？

GRU 有两个门控机制：

| 门名称 | 数学公式 | 功能解释 |
|--------|-----------|----------|
| 更新门 $z_t$ | $z_t = \sigma(W_z [h_{t-1}, x_t] + b_z)$ | 控制当前隐藏状态中，保留多少旧状态 $h_{t-1}$，引入多少新状态 |
| 重置门 $r_t$ | $r_t = \sigma(W_r [h_{t-1}, x_t] + b_r)$ | 控制在生成候选状态 $\tilde{h}_t$ 时，遗忘多少旧状态信息 |

✅ 总结：
- **更新门**决定记忆多少旧信息
- **重置门**决定整合多少旧信息用于当前判断

---

### ❓2. GRU 为什么比 RNN 强？它解决了什么问题？

GRU 比传统 RNN 更强的原因有：

- ✅ **门控机制**解决了 RNN 的梯度消失问题
- ✅ 能 **动态记忆/遗忘** 长期依赖信息
- ✅ 相比 RNN 更容易收敛，性能稳定

它的改进点：

- RNN 的隐藏状态简单叠加，容易丢失远程依赖
- GRU 通过门控控制记忆流动，**保留有用信息、抑制无关信息**

---

### ❓3. GRU 和 LSTM 各自适合什么任务？

| 模型 | 特性 | 更适合的任务类型 |
|------|------|------------------|
| **GRU** | 简洁高效，训练快，参数少 | 资源受限、快速迭代、文本分类、语音识别 |
| **LSTM** | 有独立记忆单元，长期依赖建模能力强 | 长序列任务，如翻译、语言建模、对话生成 |

✅ 经验法则：
- 如果 **速度/内存重要**，用 GRU
- 如果任务 **依赖长距离上下文**，优先用 LSTM

---

### ❓4. 为什么 GRU 没有输出门也能工作？

- LSTM 使用输出门 $o_t$ 控制 $C_t$ 对 $h_t$ 的影响
- GRU 中没有显式记忆单元 $C_t$，其状态更新为：
  
  $$
  h_t = (1 - z_t) \cdot h_{t-1} + z_t \cdot \tilde{h}_t
  $$

- 更新门 $z_t$ 本身已起到“输出控制”的作用

✅ 结论：
> GRU 的结构已将“记忆更新 + 输出调节”整合在一起，**无需单独的输出门**

---


## 📚 七、PyTorch 中使用方式

```python
rnn = nn.GRU(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=1)
output, hn = rnn(input_seq, h0)


In [6]:
import torch
from torch import nn
import time
import pandas as pd

# ---------------------
# 手写 GRU
# ---------------------
class GRUCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.W_xz = nn.Linear(input_size, hidden_size)
        self.W_hz = nn.Linear(hidden_size, hidden_size)
        self.W_xr = nn.Linear(input_size, hidden_size)
        self.W_hr = nn.Linear(hidden_size, hidden_size)
        self.W_xh = nn.Linear(input_size, hidden_size)
        self.W_hh = nn.Linear(hidden_size, hidden_size)

    def forward(self, x_t, h_prev):
        z_t = torch.sigmoid(self.W_xz(x_t) + self.W_hz(h_prev))
        r_t = torch.sigmoid(self.W_xr(x_t) + self.W_hr(h_prev))
        h_tilde = torch.tanh(self.W_xh(x_t) + self.W_hh(r_t * h_prev))
        h_t = (1 - z_t) * h_prev + z_t * h_tilde
        return h_t

class ManualGRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.cell = GRUCell(input_size, hidden_size)

    def forward(self, inputs, h0=None):
        seq_len, batch_size, _ = inputs.shape
        hidden_size = self.cell.hidden_size
        if h0 is None:
            h0 = torch.zeros(batch_size, hidden_size, device=inputs.device)
        outputs = []
        h = h0
        for t in range(seq_len):
            h = self.cell(inputs[t], h)
            outputs.append(h.unsqueeze(0))
        return torch.cat(outputs, dim=0), h.unsqueeze(0)

In [7]:
# ---------------------
# 模拟输入
# ---------------------
seq_len = 50
batch_size = 64
input_size = 32
hidden_size = 64
x = torch.randn(seq_len, batch_size, input_size)

# ---------------------
# 模型定义与包装
# ---------------------
manual_gru = ManualGRU(input_size, hidden_size)
builtin_gru = nn.GRU(input_size, hidden_size)

class GRUWrapper(nn.Module):
    def __init__(self, gru):
        super().__init__()
        self.gru = gru

    def forward(self, x):
        return self.gru(x)[0]

builtin_gru_wrapped = GRUWrapper(builtin_gru)

# ---------------------
# Benchmark 函数
# ---------------------
def benchmark(model, name, inputs, repeat=10):
    model.eval()
    with torch.no_grad():
        start = time.time()
        for _ in range(repeat):
            _ = model(inputs)
        end = time.time()
    return name, end - start

# ---------------------
# Benchmark 运行
# ---------------------
results = [
    benchmark(builtin_gru_wrapped, "nn.GRU", x),
    benchmark(manual_gru, "Manual GRU", x)
]

df = pd.DataFrame(results, columns=["Model", "Time (s)"])
print(df)

# ---------------------
# 输出误差与参数量对比
# ---------------------
out_builtin, _ = builtin_gru(x)
out_manual, _ = manual_gru(x)

max_diff = (out_builtin - out_manual).abs().max().item()
mean_diff = (out_builtin - out_manual).abs().mean().item()

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

n_builtin = count_parameters(builtin_gru)
n_manual = count_parameters(manual_gru)

print(f"\n🔍 Output max diff: {max_diff:.6f}")
print(f"🔍 Output mean diff: {mean_diff:.6f}")
print(f"🧠 Builtin GRU params: {n_builtin}")
print(f"🧠 Manual  GRU params: {n_manual}")


        Model  Time (s)
0      nn.GRU  0.076545
1  Manual GRU  0.124358

🔍 Output max diff: 1.447857
🔍 Output mean diff: 0.304520
🧠 Builtin GRU params: 18816
🧠 Manual  GRU params: 18816


# 🧠 LSTM：长短期记忆网络（Long Short-Term Memory）

---

## 📌 一、为什么要引入 LSTM？

传统 RNN 存在两个核心问题：

- **梯度消失或爆炸**：长序列中早期信息在传播过程中几乎消失，导致学习失败。
- **难以建模长期依赖**：RNN 容易被“短期上下文”干扰，难以保留长期记忆。

✅ LSTM 通过引入“**门控机制**”来控制信息的保留、遗忘与更新，缓解这些问题。

---

## 🧠 二、LSTM 的结构公式

LSTM 的每个时间步包含 4 个主要门：

> 所有门都依赖于前一时刻的隐藏状态 $h_{t-1}$ 和当前输入 $x_t$

- **遗忘门** $f_t$：
  $$
  f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)
  $$
  控制保留多少旧记忆 $C_{t-1}$

- **输入门** $i_t$：
  $$
  i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)
  $$
  控制是否写入新的记忆内容

- **候选记忆** $\tilde{C}_t$：
  $$
  \tilde{C}_t = \tanh(W_c \cdot [h_{t-1}, x_t] + b_c)
  $$

- **更新记忆单元**：
  $$
  C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t
  $$

- **输出门** $o_t$：
  $$
  o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)
  $$

- **当前隐藏状态** $h_t$：
  $$
  h_t = o_t \odot \tanh(C_t)
  $$

---

## 🏗️ 三、LSTM 单元结构图（信息流）

                         ┌─────────────┐
                         │  xₜ         │
                         └────┬────────┘
                              │
                              ▼
                         ┌─────────────┐
                         │ 拼接 [hₜ₋₁, xₜ] │
                         └────┬────────┘
                              │
            ┌────────────────┼─────────────────────┐
            ▼                ▼                     ▼
        ┌───────┐       ┌────────┐            ┌────────┐
        │ fₜ    │       │ iₜ     │            │ oₜ     │
        │sigmoid│       │sigmoid │            │sigmoid │
        └──┬────┘       └──┬─────┘            └──┬─────┘
           │               │                    │
           │               ▼                    │
     ┌─────▼─────┐   ┌─────────────┐            │
     │ Cₜ₋₁ × fₜ │   │  C̃ₜ = tanh(·) │◄────────┘
     └─────┬─────┘   └─────────────┘
           │               │
           └─────┬─────────┘
                 ▼
           ┌────────────┐
           │ Cₜ = fₜ·Cₜ₋₁ + iₜ·C̃ₜ │
           └────┬───────┘
                ▼
          ┌────────────┐
          │ hₜ = oₜ × tanh(Cₜ) │
          └────────────┘



---

## ✏️ 四、与 RNN 的区别

| 模型 | 状态结构 | 是否有记忆单元 C | 是否有门控 | 长期依赖建模能力 |
|------|----------|------------------|------------|------------------|
| RNN  | $h_t$    | ❌ 无             | ❌ 无       | 弱               |
| LSTM | $h_t, C_t$ | ✅ 有             | ✅ 有       | 强               |

---

## 🧠 五、LSTM 的优点与缺点

### ✅ 优点：
- 缓解梯度消失，能建模长依赖
- 学习何时记、何时忘，鲁棒性强

### ❌ 缺点：
- 参数量大，计算开销高
- 每步都要计算多个门，速度慢于 GRU/Transformer

---

## 📚 六、常见应用场景

- 文本生成、机器翻译（早期 Seq2Seq）
- 时间序列预测（如股票/天气）
- 情感分析、语音识别

---

## 💬 七、面试常问

- 为什么 RNN 不能捕捉长依赖？LSTM 怎么解决的？
- LSTM 有几个门？各自作用？
- LSTM 的参数量和 RNN 相比有什么变化？
- LSTM 中的 $C_t$ 和 $h_t$ 各代表什么？

---

## 📌 八、梯度裁剪（补充）

由于 LSTM 仍可能在长序列中产生梯度爆炸，因此训练时经常使用：

```python
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)


In [8]:
import torch
from torch import nn
import time
import pandas as pd

# ---------------------
# 手写 LSTM
# ---------------------
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.W_xi = nn.Linear(input_size, hidden_size)
        self.W_hi = nn.Linear(hidden_size, hidden_size)
        self.W_xf = nn.Linear(input_size, hidden_size)
        self.W_hf = nn.Linear(hidden_size, hidden_size)
        self.W_xo = nn.Linear(input_size, hidden_size)
        self.W_ho = nn.Linear(hidden_size, hidden_size)
        self.W_xc = nn.Linear(input_size, hidden_size)
        self.W_hc = nn.Linear(hidden_size, hidden_size)

    def forward(self, x_t, h_prev, c_prev):
        i_t = torch.sigmoid(self.W_xi(x_t) + self.W_hi(h_prev))
        f_t = torch.sigmoid(self.W_xf(x_t) + self.W_hf(h_prev))
        o_t = torch.sigmoid(self.W_xo(x_t) + self.W_ho(h_prev))
        c_tilde = torch.tanh(self.W_xc(x_t) + self.W_hc(h_prev))
        c_t = f_t * c_prev + i_t * c_tilde
        h_t = o_t * torch.tanh(c_t)
        return h_t, c_t

class ManualLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.cell = LSTMCell(input_size, hidden_size)

    def forward(self, inputs, h0=None, c0=None):
        seq_len, batch_size, _ = inputs.shape
        hidden_size = self.cell.hidden_size
        if h0 is None:
            h0 = torch.zeros(batch_size, hidden_size, device=inputs.device)
        if c0 is None:
            c0 = torch.zeros(batch_size, hidden_size, device=inputs.device)
        outputs = []
        h, c = h0, c0
        for t in range(seq_len):
            h, c = self.cell(inputs[t], h, c)
            outputs.append(h.unsqueeze(0))
        return torch.cat(outputs, dim=0), (h, c)

In [9]:
# ---------------------
# Benchmark Setup
# ---------------------
def benchmark(model, name, inputs, repeat=10):
    model.eval()
    with torch.no_grad():
        start = time.time()
        for _ in range(repeat):
            _ = model(inputs)
        end = time.time()
    return name, end - start

# ---------------------
# 模拟输入数据
# ---------------------
seq_len = 50
batch_size = 64
input_size = 32
hidden_size = 64
x = torch.randn(seq_len, batch_size, input_size)

# ---------------------
# 初始化模型
# ---------------------
manual_lstm = ManualLSTM(input_size, hidden_size)
builtin_lstm = nn.LSTM(input_size, hidden_size)

# 包一层 wrapper 以统一接口（仅返回 output）
class LSTMWrapper(nn.Module):
    def __init__(self, lstm):
        super().__init__()
        self.lstm = lstm

    def forward(self, x):
        return self.lstm(x)[0]

builtin_lstm_wrapped = LSTMWrapper(builtin_lstm)

# ---------------------
# 执行 Benchmark
# ---------------------
results = [
    benchmark(builtin_lstm_wrapped, "nn.LSTM", x),
    benchmark(manual_lstm, "Manual LSTM", x)
]

# 展示结果
df = pd.DataFrame(results, columns=["Model", "Time (s)"])
print(df)

# =======================
# 对比输出误差 & 参数量
# =======================
out_builtin, _ = builtin_lstm(x)
out_manual, _ = manual_lstm(x)

# 输出误差（最大值、平均值）
max_diff = (out_builtin - out_manual).abs().max().item()
mean_diff = (out_builtin - out_manual).abs().mean().item()

# 参数量统计
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

n_builtin = count_parameters(builtin_lstm)
n_manual = count_parameters(manual_lstm)

print(f"🔍 Output max diff: {max_diff:.6f}")
print(f"🔍 Output mean diff: {mean_diff:.6f}")
print(f"🧠 Builtin LSTM params: {n_builtin}")
print(f"🧠 Manual  LSTM params: {n_manual}")


         Model  Time (s)
0      nn.LSTM  0.077001
1  Manual LSTM  0.152505
🔍 Output max diff: 0.829677
🔍 Output mean diff: 0.148258
🧠 Builtin LSTM params: 25088
🧠 Manual  LSTM params: 25088
